In [2]:
# -*- coding: utf-8 -*-
# 埼玉千葉神奈川記事反映本番用
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import requests
import urllib.request as req
import glob
import json
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
import os
from dotenv import load_dotenv
from selenium.webdriver.common.by import By
from wordpress_xmlrpc import Client, WordPressPost, methods
from src.utils import *
print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()






ライブラリの読み込み完了


True

In [4]:
class Blog():
    def __init__(self):
        WORDPRESS_ID = os.environ['WORDPRESS_ID']
        WORDPRESS_PW = os.environ['WORDPRESS_PW']
        WORDPRESS_URL = os.environ['WORDPRESS_URL']
        self.wp = Client(WORDPRESS_URL, WORDPRESS_ID, WORDPRESS_PW)
        
    def add_target_date (self, target_day_number:int):
        week_list = [ '(日)','(月)', '(火)', '(水)', '(木)', '(金)', '(土)','(日)']
        target_date:datetime = datetime.date.today() + datetime.timedelta(days=target_day_number)
        #datetime(2023,3,3)
        self.target_date:datetime = datetime.date.today() + datetime.timedelta(days=target_day_number)
        #3月3日(火)
        self.target_date_string_jp:str = target_date.strftime('%m').lstrip('0') + '月' + target_date.strftime('%d').lstrip('0') + '日'  +week_list[target_date.isoweekday()]
        #2023-03-03
        self.target_date_string_sql:str = target_date.strftime('%Y-%m-%d')
        print(f'インスタンに日付:{self.target_date_string_jp}など三つの変数が追加されました')
    
    def upload_image(self,in_image_file_name, out_image_file_name):
        if os.path.exists(in_image_file_name):
            with open(in_image_file_name, 'rb') as f:
                binary = f.read()

            data = {
                "name": out_image_file_name,
                "type": 'image/png',
                "overwrite": True,
                "bits": binary
            }

            media_id = self.wp.call(media.UploadFile(data))['id']
            print(in_image_file_name.split('/')
                [-1], 'Upload Success : id=%s' % media_id)
            return media_id
        else:
            print(in_image_file_name.split('/')[-1], 'NO IMAGE!!')

    def post_line(self,message):
        url = "https://notify-api.line.me/api/notify"
        token = os.environ['LINE_TOKEN']
        headers = {"Authorization" : "Bearer "+ token}
        payload = {"message" :  message}
        #imagesフォルダの中のgazo.jpg
        #print('image_path',image_path)
        #files = {"imageFile":open(image_path,'rb')}
        post = requests.post(url ,headers = headers ,params=payload) 


    def get_post_list(self) -> list[any]:
        '''投稿一覧を取得する関数
        一回で100記事まで取得できる'''
        post_list:list[WordPressPost] = self.wp.call(methods.posts.GetPosts({"number": 50, "offset":0}))
        self.post_list = post_list
        return self.post_list

    def generate_thumbnail(self) :
        image_path = r'image\image_source\千葉.jpg'  # mac
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        write_image_context = f'''{self.target_date_string_jp }\n{self.prefecture}\n取材予定まとめ'''
        font_path = r"font/LightNovelPOPv2.otf"
        font = ImageFont.truetype(font_path, size=240)
        draw.multiline_text((130, 20), write_image_context, fill=(255, 255, 255),align='center',font=font, spacing=50, stroke_width=5, stroke_fill=(55, 55, 55))
        thumbnail_image_path = fr'image\thumbnail_{self.target_date}_{self.prefecture}.jpg'
        image.save(thumbnail_image_path)
        self.thumbnail_image_path = thumbnail_image_path

    def post_blog(self):
        title = f"【{self.prefecture}】{self.target_date_string_jp} パチンコスロットイベント取材まとめ"
        # Blog Content (html)
        body = f'''
        テスト_{self.prefecture}_{self.target_date_string_jp}_1
        '''
        # publish or draft
        status = 'draft' # "publish"　or# 'draft'

        # Category keyword
        cat1 = '取材予定まとめ'
        cat2 = ''
        cat3 = ''

        # Tag keyword
        tag1 = f'{self.prefecture}'
        tag2 = f'{self.target_date}'

        slug = f"pledge_report_{self.target_date}_{self.prefecture}"

        # Post
        post = WordPressPost()
        post.title = title
        post.content = body
        post.post_status = status
        post.terms_names = {
            "category": [cat1],
            "post_tag": [tag1],
        }
        post.slug = slug

        #サムネイル関連
        output_thumbnail_path = f'thumbnail_{self.target_date}_{self.prefecture}.jpg'
        media_id = self.upload_image(self.thumbnail_image_path, output_thumbnail_path)
        post.thumbnail = media_id

        #Post Time
        #post.date = datetime.datetime.now()

        self.wp.call(NewPost(post))
        print('記事書き込み完了')
        # except Exception as e:
    #     post_line(f'{tomorrow}分ブログ投稿失敗\n{e}')
    
    def wp_update_post(self,content_id, changed_content):

        # URL, User, Password設定
        WP_URL: str = "https://slotana777.com"
        WP_USER: str = os.getenv('WP_USER')
        WP_API_PASSWORD: str = os.getenv('REST_API_PW')
        API_URL = f"{WP_URL}/wp-json/wp/v2/"
        url = f'{WP_URL}/wp-json/wp/v2/posts/{content_id}'


        params = {'content':changed_content}

        res = requests.post(
            url,
            params=params,
            auth=( WP_USER, WP_API_PASSWORD),
            )

        return res




In [5]:
blog= Blog()

In [6]:
post_list :list = blog.get_post_list()

In [8]:
post.id

'37455'

In [11]:
post_title_contentid_dict['【長野県】3月28日(火) パチンコスロットイベント取材まとめ']

37446

In [13]:
if '【長野県】3月28日(火) パチンコスロットイベント取材まとめ' in post_title_contentid_dict

False

In [10]:
post_title_contentid_dict:dict[str:int] = {}
for post in post_list:
    post_title_contentid_dict[post.title] = int(post.id)
    print(post.title,post.id)
    #break
    
title = '【長野県】3月28日(火) パチンコスロットイベント取材まとめ'
if title in post_title_contentid_dict:
    update_content_id = post_title_contentid_dict[title]
    print(update_content_id )
else:
    print('新しい記事を作成')

【長野県】3月29日(水) パチンコスロットイベント取材まとめ 37455
タイトル 37449
Pythonで自動投稿 -確認- 37451
【長野県】3月28日(火) パチンコスロットイベント取材まとめ 37446
【3/29(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37410
【3/28(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37382
【3/27(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37355
【3/26(日)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37324
【3/25(土)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37296
【3/24(金)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37267
【3/23(木)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37240
【3/22(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37213
【3/21(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37186
【3/20(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37159
【3/19(日)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37131
【3/18(土)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37104
【3/17(金)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37077
【3/16(木)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37050
【3/15(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 37023
【3/14(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 36996
【3/13(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 36969
【3/12(日)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 36942
【3/11(土)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】 369

In [19]:
blog.wp_update_post(post.title,post.content)

<Response [404]>

In [5]:
prefecture_list = ['長野県','静岡県','山梨県','愛知県']
for target_day_number  in range(3,6):
    for prefecture in prefecture_list:
        blog = Blog()
        print(prefecture,target_day_number)
        blog.add_target_date (target_day_number)
        blog.prefecture = prefecture
        blog.add_target_date(target_day_number)
        title = f"【{blog.prefecture}】{blog.target_date_string_jp } パチンコスロットイベント取材まとめ"
        print(title)
        blog.generate_thumbnail()
        blog.post_blog()
        break
    break

長野県 3
インスタンに日付:3月29日(水)など三つの変数が追加されました
インスタンに日付:3月29日(水)など三つの変数が追加されました
【長野県】3月29日(水) パチンコスロットイベント取材まとめ
image\thumbnail_2023-03-29_長野県.jpg Upload Success : id=37454
記事書き込み完了


In [29]:
blog.get_post_list()
for post in blog.post_list:
    print(post.title,post.id)
    break

【長野県】3月28日(火) パチンコスロットイベント取材まとめ 37446


In [1]:
import requests
import json
import base64

from dotenv import load_dotenv
from selenium.webdriver.common.by import By
from wordpress_xmlrpc import Client, WordPressPost, methods
from src.utils import *
print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()

# URL, User, Password設定
WP_URL: str = "https://slotana777.com"
WP_USER: str = "python_rest_api"
WP_API_PASSWORD: str = os.getenv('REST_API_PW')
API_URL = f"{WP_URL}/wp-json/wp/v2/"


読み込み完了
ライブラリの読み込み完了


In [5]:
from urllib.parse import urljoin

tag_url = urljoin(WP_URL, '/wp-json/wp/v2/tags/')
def wp_tag_add(tagname):
    r = ""
    post = {
        'name': tagname,
    }
    res = requests.post(
        tag_url ,
        json=post,
        auth=(WP_USER, WP_API_PASSWORD),
        )
    data = json.loads(res.text)
    if 'code' in data:
        if data['code'] == "term_exists":
            r = data['data']['term_id']
            print(r)
    else:
        r = data['id']
        print(r)
    return r
r = wp_tag_add('奈良県')

919


919